- pca to remove unnecessary c vars
- try 2nd layer again

In [1]:
import os
import sys
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn import preprocessing
from lightgbm import LGBMClassifier
from sklearn.metrics import log_loss, mean_squared_error
from sklearn.decomposition import PCA
from tqdm._tqdm_notebook import tqdm_notebook
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import StratifiedKFold, KFold
sys.path.append('../input/multilabelstraifier/')
sys.path.append('../input/enhance/')
from ml_stratifiers import MultilabelStratifiedKFold
import enhance
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  del sys.path[0]


ModuleNotFoundError: No module named 'enhance'

In [2]:
DIR = "/kaggle/input/lish-moa/"
train = pd.read_csv(DIR+"train_features.csv")
test = pd.read_csv(DIR+"test_features.csv")
nontargets = pd.read_csv(DIR+"train_targets_nonscored.csv")
targets = pd.read_csv(DIR+"train_targets_scored.csv")
sub = pd.read_csv(DIR+"sample_submission.csv")

In [3]:
target_feats = [ i for i in targets.columns if i != "sig_id"]
g_feats = [i for i in train.columns if "g-" in i]
c_feats = [i for i in train.columns if "c-" in i]

In [4]:
noncons_train_index = train[train.cp_type=="ctl_vehicle"].index
cons_train_index = train[train.cp_type!="ctl_vehicle"].index
noncons_test_index = test[test.cp_type=="ctl_vehicle"].index
cons_test_index = test[test.cp_type!="ctl_vehicle"].index

In [5]:
train = train[train.index.isin(cons_train_index)].copy().reset_index(drop=True)
n_targets = targets[targets.index.isin(cons_train_index)].copy().reset_index(drop=True)
n_nontargets = nontargets[nontargets.index.isin(cons_train_index)].copy().reset_index(drop=True)

In [6]:
c_num = 10
pca_c_cols = ["pca-c"+str(i) for i in range(c_num)]
pca = PCA(n_components=c_num)
pca_c_train = pca.fit_transform(train[c_feats])
pca_c_test = pca.transform(test[c_feats])
pca_c_train = pd.DataFrame(pca_c_train, columns=pca_c_cols)
pca_c_test = pd.DataFrame(pca_c_test, columns=pca_c_cols)

#g_num = 400
#pca_g_cols = ["pca-g"+str(i) for i in range(g_num)]
#pca = PCA(n_components=g_num)
#pca_g_train = pca.fit_transform(train[g_feats])
#pca_g_test = pca.transform(test[g_feats])
#pca_g_train = pd.DataFrame(pca_g_train, columns=pca_g_cols)
#pca_g_test = pd.DataFrame(pca_g_test, columns=pca_g_cols)

train = pd.concat([train, pca_c_train], axis=1)
test = pd.concat([test, pca_c_test], axis=1)
#train = pd.concat([train, pca_g_train], axis=1)
#test = pd.concat([test, pca_g_test], axis=1)

# preprocess

In [7]:
# normalization by ctl group
#train_ctl = train[train.index.isin(noncons_train_index)].copy().reset_index(drop=True)
#test_ctl = test[test.index.isin(noncons_test_index)].copy().reset_index(drop=True)
#ctl_df = pd.concat([train_ctl, test_ctl])

#ctl_group_data = ctl_df.groupby(["cp_dose", "cp_time"]).agg({"mean"}).reset_index()
#mean_g_feats = ["mean-g-" + str(i) for i in range(g_num)]
#mean_c_feats = ["mean-c-" + str(i) for i in range(c_num)]
#columns = ["cp_dose", "cp_time"] + mean_g_feats + mean_c_feats
#ctl_group_data.columns = columns

#train_cons = train_feat[train_feat.index.isin(cons_train_index)].copy().reset_index(drop=True)
#test_cons = test_feat[test_feat.index.isin(cons_test_index)].copy().reset_index(drop=True)
#n_train_score = train_score[train_score.index.isin(cons_train_index)].copy().reset_index(drop=True)
#n_train_nonscore = train_nonscore[train_nonscore.index.isin(cons_train_index)].copy().reset_index(drop=True)

#n_train = pd.merge(train, ctl_group_data, on=["cp_time", "cp_dose"], how="left")
#n_test = pd.merge(test, ctl_group_data, on=["cp_time", "cp_dose"], how="left")

#for i in range(g_num):
#    n_train["diff-g-"+str(i)] = n_train["pca-g"+str(i)] - n_train["mean-g-"+str(i)]
#    n_test["diff-g-"+str(i)] = n_test["pca-g"+str(i)] - n_test["mean-g-"+str(i)]
    
#for i in range(c_num):
#    n_train["diff-c-"+str(i)] = n_train["pca-c"+str(i)] - n_train["mean-c-"+str(i)]
#    n_test["diff-c-"+str(i)] = n_test["pca-c"+str(i)] - n_test["mean-c-"+str(i)]

In [8]:
categoricals = ["cp_dose"]

def encoding(tr, te):
    for f in categoricals:
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(tr[f]))
        tr[f] = lbl.transform(list(tr[f]))
        te[f] = lbl.transform(list(te[f])) 
        
    return tr, te

n_train, n_test = encoding(train, test)

# feature engineering

In [9]:
def fe(df, remove_features):
    df.drop(remove_features, axis=1, inplace=True)
    return df

remove_features = ["cp_type", "sig_id"] + c_feats
        
n_train = fe(n_train, remove_features)
n_test = fe(n_test, remove_features)

print(train.shape, test.shape)

(21948, 784) (3982, 784)


# 1st model

In [10]:
def modelling_lgb(new_train, target_train, new_test, target, ind):
    
    X_train = new_train.copy()
    y_train = target_train[target].copy()
    
    params = {'objective': 'binary', 'boosting_type': 'gbdt', 'tree_learner': 'serial', 'learning_rate': 0.01, 
               "num_leaves": 10, 'random_seed':44, 'max_depth': 5} 
        
    n_folds=4
    if target not in ["erbb2_inhibitor", "atp-sensitive_potassium_channel_antagonist"]:
        skf=StratifiedKFold(n_splits = n_folds, shuffle=True, random_state=0)
    else:
        skf=KFold(n_splits = n_folds, shuffle=True, random_state=0)

    models = []

    valid = np.zeros([X_train.shape[0]])
    for i , (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
        X_train2 = X_train.iloc[train_index,:]
        y_train2 = y_train.iloc[train_index]

        X_valid2 = X_train.iloc[test_index,:]
        y_valid2 = y_train.iloc[test_index]
        
        lgb_train = lgb.Dataset(X_train2, y_train2)
        lgb_eval = lgb.Dataset(X_valid2, y_valid2, reference=lgb_train)
        
        clf = lgb.train(params, lgb_train,valid_sets=[lgb_train, lgb_eval], 
               num_boost_round=10000,early_stopping_rounds=25,verbose_eval = 0) 

        valid_predict = clf.predict(X_valid2, num_iteration = clf.best_iteration)
        valid[test_index] = valid_predict
        models.append(clf)

    if new_test is not None:
        X_test = new_test.copy()
        pred_value = np.zeros(X_test.shape[0])
        for model in models:
            pred_value += model.predict(X_test, num_iteration = clf.best_iteration) / n_folds
    else:
        pred_value = None
            
    score = log_loss(y_train, valid)
            
    return valid, pred_value, score

train_checkscore = targets.copy()
target_list = []
log_loss_list = []

for ind, target in enumerate(target_feats):
    print(ind, target)
    valid, pred_value, score = modelling_lgb(n_train, n_targets, n_test, target, ind)
    train_checkscore.loc[cons_train_index, target] = valid
    train_checkscore.loc[noncons_train_index, target] = 0
    print("oof log_loss= {}, all log_loss= {}".format(score, log_loss(targets[target], train_checkscore[target])))
    target_list.append(target)
    log_loss_list.append(score)
    sub[target] = pred_value
    sub.loc[noncons_test_index, target] = 0

0 5-alpha_reductase_inhibitor
oof log_loss= 0.006037989903573974, all log_loss= 0.005564869505485993
1 11-beta-hsd1_inhibitor
oof log_loss= 0.00656412505243258, all log_loss= 0.006049778141042753
2 acat_inhibitor
oof log_loss= 0.008451011471681083, all log_loss= 0.007788813293879996
3 acetylcholine_receptor_agonist
oof log_loss= 0.048828049779998044, all log_loss= 0.04500201715677328
4 acetylcholine_receptor_antagonist
oof log_loss= 0.07045251154881163, all log_loss= 0.06493204516138908
5 acetylcholinesterase_inhibitor
oof log_loss= 0.022247860099987655, all log_loss= 0.02050457854516381
6 adenosine_receptor_agonist
oof log_loss= 0.017186472952090755, all log_loss= 0.01583978787068488
7 adenosine_receptor_antagonist
oof log_loss= 0.027584014015177186, all log_loss= 0.025422606013484117
8 adenylyl_cyclase_activator
oof log_loss= 0.0037737897631559374, all log_loss= 0.0034780859041634495
9 adrenergic_receptor_agonist
oof log_loss= 0.06162904055340068, all log_loss= 0.05679995725480978
10

In [11]:
# local score
scores = []
for target_col in target_feats:
    scores.append(log_loss(targets[target_col], train_checkscore[target_col]))
print(np.mean(scores))

0.015974111427706824


# 2nd model

In [12]:
first_layer_train = train_checkscore[train_checkscore.index.isin(cons_train_index)].copy()
first_layer_test = sub[sub.index.isin(cons_test_index)].copy()

first_layer_train.drop("sig_id", axis=1, inplace=True)
first_layer_test.drop("sig_id", axis=1, inplace=True)

#n_train_feat2 = pd.merge(n_train, first_layer_train_feat, on="sig_id", how="outer") 
#n_test_feat2 = pd.merge(n_test, first_layer_test_feat, on="sig_id", how="outer")

In [13]:
final = pd.read_csv(DIR+"sample_submission.csv")

In [14]:
train_checkscore2 = targets.copy()
target_list2 = []
log_loss_list2 = []

for ind, target in enumerate(target_feats):
    print(ind, target)
    valid, pred_value, score = modelling_lgb(first_layer_train, n_targets, first_layer_test, target, ind)
    train_checkscore2.loc[cons_train_index, target] = valid
    train_checkscore2.loc[noncons_train_index, target] = 0
    print("oof log_loss= {}, all log_loss= {}".format(score, log_loss(targets[target], train_checkscore2[target])))
    target_list2.append(target)
    log_loss_list2.append(score)
    final.loc[cons_test_index, target] = pred_value
    final.loc[noncons_test_index, target] = 0

0 5-alpha_reductase_inhibitor
oof log_loss= 0.006079987563625124, all log_loss= 0.00560357634359814
1 11-beta-hsd1_inhibitor
oof log_loss= 0.006623016769963944, all log_loss= 0.006104055264431449
2 acat_inhibitor
oof log_loss= 0.008518444820570233, all log_loss= 0.0078509627497219
3 acetylcholine_receptor_agonist
oof log_loss= 0.048609980880781986, all log_loss= 0.044801035540917314
4 acetylcholine_receptor_antagonist
oof log_loss= 0.06970999105625625, all log_loss= 0.06424770654668321
5 acetylcholinesterase_inhibitor
oof log_loss= 0.021850029795857144, all log_loss= 0.020137921137124145
6 adenosine_receptor_agonist
oof log_loss= 0.017246103197969615, all log_loss= 0.015894745653356808
7 adenosine_receptor_antagonist
oof log_loss= 0.027478250137933462, all log_loss= 0.025325129504802445
8 adenylyl_cyclase_activator
oof log_loss= 0.0033197338192419343, all log_loss= 0.0030596085439121455
9 adrenergic_receptor_agonist
oof log_loss= 0.061424810611786236, all log_loss= 0.056611730213634255

In [15]:
# local score
scores = []
for target_col in target_feats:
    scores.append(log_loss(train_score[target_col], train_checkscore2[target_col]))
print(np.mean(scores))

NameError: name 'train_score' is not defined

# submission

In [16]:
final.to_csv("submission.csv", index=False)